Мне показалось, что классы RFM в предоставленной таблице не достаточно  ясно отражают специфику каждой группы и я решила дополнительно их классифицировать. Для этого по каждому показателю (Recency, Frequency, Monetary) группы получили баллы: 3 (33% лучших), 1 (33% худших), 2 (средние).

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('rfm.xlsx')

In [3]:
data.info()
data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   RFM Class  13 non-null     int64  
 1   Recency    13 non-null     float64
 2   Frequency  13 non-null     float64
 3   Monetary   13 non-null     float64
 4   Users      13 non-null     int64  
dtypes: float64(3), int64(2)
memory usage: 648.0 bytes


,RFM Class,Recency,Frequency,Monetary,Users
0,3,102.62,1.00,1061.00,578
1,4,89.72,1.00,2170.32,804
2,5,73.67,1.00,3279.72,1032
3,6,61.49,1.04,4599.98,1269
4,7,50.62,1.07,6711.21,1392
5,8,40.23,1.18,7469.94,1223
6,9,35.71,1.27,10094.50,1127
7,10,35.63,1.54,14008.70,941
8,11,39.38,2.07,17898.50,648
9,12,40.95,2.98,18291.83,442


In [4]:
# Считаем 33 и 66-й перцентили по каждому из показателей. 
# Помним, что в случае давности покупок максимальный балл присваивается группам с наименьшими значениями, 
# в случае частоты и суммы - с наибольшими.
for i in ['Recency', 'Frequency', 'Monetary']:
    p33 = round(np.percentile(data[i], 33), 2)
    p66 = round(np.percentile(data[i], 66), 2)
    name = str(i)+'_rank'
    if i == 'Recency':
        data.loc[data[i] < p33, name] = 3
        data.loc[data[i] > p66, name] = 1
    else:
        data.loc[data[i] < p33, name] = 1
        data.loc[data[i] > p66, name] = 3
    data[name].fillna(2, inplace=True)

In [5]:
#выводим совокупную оценку по трем показателям
data['RFM'] = (data['Recency_rank']*100+data['Frequency_rank']*10+data['Monetary_rank']).astype('int')

In [6]:
data

,RFM Class,Recency,Frequency,Monetary,Users,Recency_rank,Frequency_rank,Monetary_rank,RFM
0,3,102.62,1.00,1061.00,578,1.0,1.0,1.0,111
1,4,89.72,1.00,2170.32,804,1.0,1.0,1.0,111
2,5,73.67,1.00,3279.72,1032,1.0,1.0,1.0,111
3,6,61.49,1.04,4599.98,1269,1.0,1.0,1.0,111
4,7,50.62,1.07,6711.21,1392,1.0,2.0,2.0,122
5,8,40.23,1.18,7469.94,1223,2.0,2.0,2.0,222
6,9,35.71,1.27,10094.50,1127,2.0,2.0,2.0,222
7,10,35.63,1.54,14008.70,941,3.0,2.0,2.0,322
8,11,39.38,2.07,17898.50,648,2.0,3.0,3.0,233
9,12,40.95,2.98,18291.83,442,2.0,3.0,3.0,233


__Наиболее перспективны группы те,__ которые получили максимальный балл по одному-двум показателям (считаем, что группы, где все оценки - максимальные, нас полностью устраивают).  
Это __группы 11 и 12__, которые покупают часто и на большие суммы. Цель работы с ними - получить новые заказы.  
__Группа 10__ покупала недавно, с ней нужно работать над увеличением суммы заказов и частоты.  
__Группы 8 и 9__ - середнячки, в них входит бóльшее количество человек (по сравнению с группами, обсуждавшимися выше) и повышение их показателей будет иметь значимый эффект.